## Explicit Euler vs. Implicit-in-velocity Euler
Explicit Euler and implicit-in-velcoity Euler are both used to update the state of a system. However, they work a little bit differently. Explicit Euler updates the state based on the the current state of the system as you can see from the equation:
$$
y_{n+1} = y_n + \Delta t \cdot f(t_n, y_n)
$$
Where:
- $y_{n+1}$ is the state at the next time step
- $y_n$ is the current state at the current time step
- $\Delta t \cdot$ is the time step
- $f(t_n, y_n)$ is the function that describes the system's dynamics at the current time step

While, implicit-in-velocity Euler updates the velocity implicity and the positions explicitly. The equation for velocity is:
$$
v_{t+h} = v_t + h * a(v_{t+h})
$$
Where:
- $v_{t+h}$ is the velocity at the next time step (this is unknown and has to be solved for)
- $v_t$ is the current velocity
- $h$ is the time step
- $a(v_{t+h})$ is the acceleration at the next time step, which depends on the velocity at the next time step

Since $a(v_{t+h})$ is a function of $v_{t+h}$ this forms a non-linear equation. In order to solve for $v_{t+h}$ Mujoco uses a **first-order Taylor Expansion** around $v_t$ since we cannot evaluate $a(v_{t+h})$ without the next velocity. This takes the form of: 
$$
a(v_{t+h}) \approx a(v_t) + \frac{\partial v}{\partial a(v)} \cdot (v_{t+h} - v_t)
$$
Where:
- $a(v_{t+h})$ repesents the acceleration at time $t+h$ and is a function of velocity at the next time step
- $a(v_t)$ is the acceleration at the current time and is a function of the current velocity
- $\frac{\partial v}{\partial a(v)} \cdot$ is the derivative of velocity with respect to acceleration, which is used to approximate the acceleration at the next time step
- $(v_{t+h} - v_t)$ is the change in velocity over the next time step

The derivative is given by:
$$
\frac{\partial v}{\partial a(v)} = M^{-1} D
$$
Where: 
- $M^{-1}$ is the inverse of the mass matrix for the system. $M$ represents the distribution of mass in the system and how it relates to forces acting on the objects
- $D$ is the damping matrix that encodes how the forces change with velocities, excluding constraint forces. 

This simplifies to the following equation to give us the velocity update: 
$$
v_{t+h} = v_t + h \hat{M}^{-1} M a(v_t)
$$
With:
$$
\hat{M} \equiv M - hD
$$

The position is then updated using $v_{t+h}$:
$$
q_{t+h} = q_t + h * v_{t+h}
$$

The main similarities between these two algorithms are that they both predict the state of the system at the next time step based on the current conditions, and they are both first-order integration methods meaning they use the first derivative or the rate of change of the system's state to predict the next state. The main difference is that one is explicit and the other is implicit. Explicit means that it computes the next state of the system using values that have already been computed, while implicit uses values that depend on the unknown future state. Another difference is the explicit Euler is stable for smaller time steps, but can become unstable if the time step is too large. The implicit-in-velocity Euler tends to be more stable for stiff systems since it computes the velocity from the next time step. The explicit Euler is also relatively simple to implement, as we did in class, while the implicit-in-velocity Euler requires solving a non-linear equation for the velocity update which is much more difficult. Overall, both algorithms update the state of the system, explicit Euler is easier to implement, but less stable, and the implicit-in-velovity Euler is more complex, but more stable especially for systems with damping, springs, and/or stiff systems.



In [24]:
import mujoco
import mediapy as media
import numpy as np

# Load the model from the XML file
model = mujoco.MjModel.from_xml_path("trampoline.xml")  # Ensure the file path is correct
data = mujoco.MjData(model)

# Define simulation parameters
duration = 5.0  # Duration of the simulation in seconds
framerate = 60  # Frames per second (video framerate)

# Reset the simulation state and time
mujoco.mj_resetData(model, data)

# Capture frames during the simulation
frames = []
with mujoco.Renderer(model) as renderer:
    while data.time < duration:
        mujoco.mj_step(model, data)  # Step forward in time

        # Capture frames at the specified framerate
        if len(frames) < data.time * framerate:
            renderer.update_scene(data)  # Update the scene
            pixels = renderer.render()   # Render the current frame
            frames.append(pixels)        # Append to the list of frames

# Display the video
media.show_video(frames, fps=framerate)


This browser does not support the video tag.


In [25]:
# Load the model from the XML file
model = mujoco.MjModel.from_xml_path("trampoline2.xml")
data = mujoco.MjData(model)

# Set initial velocity for the ball (assuming it's the first body)
data.qvel[0:3] = [0, 0, -3]  # Set linear velocity in x, y, z directions

# After loading the model
model.opt.gravity[2] = -5.0  # Reduce gravity (default is -9.81)

# Define simulation parameters
duration = 5.0  # Duration of the simulation in seconds
framerate = 60  # Frames per second (video framerate)

# Reset the simulation state and time
mujoco.mj_resetData(model, data)

# Capture frames during the simulation
frames = []
with mujoco.Renderer(model) as renderer:
    while data.time < duration:
        mujoco.mj_step(model, data)  # Step forward in time

        # Capture frames at the specified framerate
        if len(frames) < data.time * framerate:
            renderer.update_scene(data)  # Update the scene
            pixels = renderer.render()   # Render the current frame
            frames.append(pixels)        # Append to the list of frames

# Display the video
media.show_video(frames, fps=framerate)

This browser does not support the video tag.


In [26]:
import mujoco
import numpy as np
from IPython import display
import mediapy as media  # Assuming this is your video display module

# Load the model from the XML file
model = mujoco.MjModel.from_xml_path("trampoline3.xml")
data = mujoco.MjData(model)

# Define simulation parameters
duration = 10.0  # Extended duration to see multiple bounces
framerate = 60  # Frames per second (video framerate)

# Variables to track bouncing state
last_bounce_time = -1.0
bounce_cooldown = 0.5  # Minimum time between bounces in seconds
max_height_reached = 0.0  # Track maximum height
target_height = 4.0  # Target max height for bounces

# IMPORTANT: Reset the simulation and set initial conditions
mujoco.mj_resetData(model, data)

# Set ball to start at the center with downward velocity
data.qpos[0:3] = [0.0, 0.0, 2.0]  # Lower starting height
data.qvel[0:3] = [0, 0, -2]       # Gentler initial velocity

# Ensure normal gravity
model.opt.gravity[2] = -2  # Standard gravity

# Function to apply forces to the ball
def apply_forces(model, data):
    global last_bounce_time, max_height_reached
    
    # Get ball position from sensor
    ball_pos_idx = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SENSOR, "ball_pos")
    ball_x = data.sensordata[ball_pos_idx * 3]     # X-coordinate
    ball_y = data.sensordata[ball_pos_idx * 3 + 1] # Y-coordinate
    ball_z = data.sensordata[ball_pos_idx * 3 + 2] # Z-coordinate
    
    # Update max height tracking
    if ball_z > max_height_reached:
        max_height_reached = ball_z
    
    # Get ball velocity
    ball_vel_x = data.qvel[0]  # X-component of velocity
    ball_vel_y = data.qvel[1]  # Y-component of velocity
    ball_vel_z = data.qvel[2]  # Z-component of velocity
    
    # Strong centering force
    centering_strength = 10.0
    data.qfrc_applied[0] = -ball_x * centering_strength
    data.qfrc_applied[1] = -ball_y * centering_strength
    
    # Add damping to horizontal velocity
    damping = 1.0
    data.qfrc_applied[0] -= ball_vel_x * damping
    data.qfrc_applied[1] -= ball_vel_y * damping
    
    # Reset vertical force
    data.qfrc_applied[2] = 0.0
    
    # Apply upward impulse when conditions are met
    if (ball_z < 1.2 and ball_vel_z < -0.5 and 
        (data.time - last_bounce_time) > bounce_cooldown):
        
        # Calculate adaptive force based on previous max height
        # If it went too high, reduce force; if too low, increase force
        base_force = 80.0
        height_factor = 1.0
        
        if max_height_reached > 0.5:  # Only adjust if we have meaningful data
            height_factor = target_height / max_height_reached
            # Limit the adjustment factor to prevent extreme changes
            height_factor = max(0.5, min(1.5, height_factor))
        
        bounce_force = base_force * height_factor
        
        # Apply the calculated force
        data.qfrc_applied[2] = bounce_force
        
        # Record this bounce time
        last_bounce_time = data.time
        
        # Reset max height tracking for next bounce
        max_height_reached = 0.0
        
        print(f"Bounce at t={data.time:.2f}, pos=({ball_x:.2f}, {ball_y:.2f}, {ball_z:.2f}), vel={ball_vel_z:.2f}, force={bounce_force:.1f}")

# Capture frames during the simulation
frames = []
with mujoco.Renderer(model) as renderer:
    while data.time < duration:
        # Apply forces
        apply_forces(model, data)
        
        # Step forward in time
        mujoco.mj_step(model, data)
        
        # Capture frames at the specified framerate
        if len(frames) < data.time * framerate:
            renderer.update_scene(data)
            pixels = renderer.render()
            frames.append(pixels)
            
            # Print ball position every second
            if len(frames) % framerate == 0:
                ball_pos_idx = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SENSOR, "ball_pos")
                ball_z = data.sensordata[ball_pos_idx * 3 + 2]
                print(f"Time {data.time:.2f}: Ball height = {ball_z:.2f}")

# Display the video
media.show_video(frames, fps=framerate)

Bounce at t=0.00, pos=(0.00, 0.00, 0.00), vel=-2.00, force=80.0
Bounce at t=0.50, pos=(-0.00, -0.00, 0.78), vel=-2.06, force=120.0
Time 0.98: Ball height = 1.40
Time 1.98: Ball height = 2.29
Bounce at t=2.97, pos=(-0.00, -0.00, 1.20), vel=-2.09, force=120.0
Time 2.98: Ball height = 1.18
Time 3.98: Ball height = 1.76
Time 4.98: Ball height = 1.72
Bounce at t=5.36, pos=(-0.00, 0.00, 1.20), vel=-1.63, force=120.0
Time 5.98: Ball height = 1.09
Time 6.98: Ball height = 1.72
Bounce at t=7.55, pos=(-0.01, 0.02, 1.20), vel=-1.49, force=120.0
Time 7.98: Ball height = 0.81
Time 8.98: Ball height = 1.25
Bounce at t=9.11, pos=(-0.01, 0.02, 1.20), vel=-0.55, force=120.0
Time 9.98: Ball height = 0.99


This browser does not support the video tag.


In [27]:
import mujoco
import mediapy as media
import numpy as np

# Load the model from the XML file
model = mujoco.MjModel.from_xml_path("tramphumanoid.xml")  # Ensure the file path is correct
data = mujoco.MjData(model)

# Define simulation parameters
duration = 5.0  # Duration of the simulation in seconds
framerate = 60  # Frames per second (video framerate)

# Reset the simulation state and time
mujoco.mj_resetData(model, data)

# Capture frames during the simulation
frames = []
with mujoco.Renderer(model) as renderer:
    while data.time < duration:
        mujoco.mj_step(model, data)  # Step forward in time

        # Capture frames at the specified framerate
        if len(frames) < data.time * framerate:
            renderer.update_scene(data)  # Update the scene
            pixels = renderer.render()   # Render the current frame
            frames.append(pixels)        # Append to the list of frames

# Display the video
media.show_video(frames, fps=framerate)

ValueError: Error: repeated name 'ground' in geom